In [ ]:
%pip install git+https://github.com/Open-Dataplatform/utils-databricks.git@v0.2.1

In [ ]:
import os
from pyspark.sql.functions import col
import pyspark.sql.functions as F

from custom_utils.dp_storage.connector import mount, unmount_if_prod
from custom_utils.dp_storage.writer import merge_and_upload
from custom_utils import adf

# Transformation

Setup

In [ ]:
mount_point = mount(dbutils)

In [ ]:
dbutils.widgets.removeAll()

# Get the parameters from adf. When developing, insert values in the text widgets.
source_config = adf.get_parameter_json('SourceConfig')
destination_config = adf.get_parameter_json('DestinationConfig')
source_folder_path = adf.get_parameter('SourceFolderPath')  # Remember that it has the format "<container>/<directory>"
source_filename = adf.get_parameter('SourceFileName')



In [ ]:
# Define key columns
timestamp_column = 'HIST_TIMESTAMP'
index_columns = [timestamp_column, 'TODO: Add all key columns']
time_resolution_egress = 'hour'

In [ ]:
# Verify that config and trigger parameters are aligned
container_from_trigger = source_folder_path.split('/')[0]
guid_from_trigger = source_folder_path.split('/')[1]
directory_from_trigger = '/'.join(source_folder_path.split('/')[1:])  # Remember that folderPath from an ADF trigger has the format "<container>/<directory>"

assert container_from_trigger == source_config['TODO: dataset_identifier']['container']
assert guid_from_trigger == source_config['TODO: dataset_identifier']['dataset']

## Read
Reads data from storage

In [ ]:
source_path = os.path.join(mount_point, directory_from_trigger, source_filename)

df = spark.read.parquet(source_path)
# df = spark.read.option("delimiter", ",").csv(source_path, header=True)

In [ ]:
df.show(3)

## Transform

Transform the data here. Follow this style guide: https://github.com/palantir/pyspark-style-guide

## Merge and upload

In [ ]:
merge_and_upload(
    df,
    destination_config['TODO: insert identifier name']['dataset'],
    timestamp_column,
    index_columns,
    time_resolution_egress,
    mount_point,
    spark
)

In [ ]:
# Always keep this at the end of your notebook
unmount_if_prod(mount_point, dbutils)